In [ ]:
import pandas as pd
import h2o
from h2o.automl import H2OAutoML

In [ ]:
df = pd.read_csv("df_ready.csv")

df_filter = df.filter(
    items = [
        "AgeMonthsInsemination",
        "Milk",
        "Fat",
        "Proteins",
        "ThiMax_dayless_5",
        "ThiMax_dayless_30",
        "ThiMax_dayless_60",
        "ThiMax_dayless_90",
        "ThiMax_dayless_120",
        "DaysOpen"
    ]
)

h2o.init(max_mem_size = "150G")

df_h2o = h2o.H2OFrame(df_filter)
df_train, df_validation, df_test = df_h2o.split_frame(ratios = [0.7, 0.2])

targets = [
    "DaysOpen"
]

for target in targets:

    X = [column for column in df_h2o.columns if column not in targets]
    y = target

    aml = H2OAutoML(
        max_runtime_secs = 23 * 3600,
        seed = 42,
        nfolds = 0,
        exclude_algos = ["StackedEnsemble"],
        verbosity = "info"
    )

    aml.train(
        x = X,
        y = y,
        training_frame = df_train,
        validation_frame = df_validation,
        leaderboard_frame = df_test
    )

    lb = aml.leaderboard  # get leader board
    
    print(f"Target: {target}")
    print(lb.head(20))

    df_lb = lb.as_data_frame()
    df_lb.to_csv(f"lb_{target}.csv")